In [20]:
########## import libraries ############
import numpy as np
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import zeros
import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Embedding
import IPython; from IPython.display import display, HTML
def dfPrint(df):
    display(HTML(df.to_html()))
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cardioid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# ############# import data ##############
# def import_data():
#     train_path = "../data/text_large_infy_mod.csv"
#     data = pd.read_csv(train_path)
#     data.reset_index(inplace=True)
#     data.drop('index', axis=1, inplace=True)
#     data.drop(['year','month','date','hour','mins','no'], axis=1, inplace=True)
#     print ('dataset loaded with shape', data.shape)
#     return data

# data = import_data()
# dfPrint(data.sample(5))

In [22]:
############ import data ##############
def import_data():
    train_path = "../data/AXISBANK.csv"
    data = pd.read_csv(train_path)
#     data.reset_index(inplace=True)
#     data.drop('index', axis=1, inplace=True)
    data.drop(['time','index'], axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape)
    return data

data = import_data()
dfPrint(data.sample(20))

dataset loaded with shape (1492, 2)


,title,label
783,BSE's transaction charge waiver on Sensex stoc...,0
290,Timeline: Developments in the $2 billion Punja...,1
350,News in Numbers: 31 banks gave working capital...,1
489,Google Alert - NON PERFORMING ASSETS,0
177,"Nifty | Stock Market Now: Nifty above 10,450, ...",1
101,Ways2Capital Closing Update : Sensex Ends Off ...,1
1243,Google Alert - Bank,0
1004,Federal Bank Share Price: Market Now: Private ...,1
1035,Sensex slips 144 points ahead of F&O expiry,0
241,Google Alert - income tax india,0


In [23]:
def clean_decription(line):
    words = list(set(line.split()))
    words = list(sorted([x.lower() for x in words]))
#     print (len(words))
#     print (words)
    words2 = words.copy()
    for word in list(set(words2).intersection(stop_words)):
        words.remove(word)
#     out = ' '.join(words)
    out = ' '.join(e for e in words if e.isalnum())
    if out == "":
        print(words)
#     table = str.maketrans({key: None for key in string.punctuation})
#     out.translate(table, string.punctuation)
    return out
data["title"] = data["title"].map(clean_decription)

In [24]:
############### define documents ################
docs = data["title"]
############# define class labels ################
labels = data["label"]
############## prepare tokenizer #################
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

########## integer encode the documents ##########
encoded_docs = t.texts_to_sequences(docs)
########## pad documents to a max length ##########
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

print(docs.sample(5))
print(vocab_size)

1033                           claim cpm exposes pnb scam
174     asks bata india leak probe quarterly results s...
285     11999 32 6 drop flipkart gb honor hot online p...
1171                           gainers losers session top
1228    agencies banks branches indian nirav overseas ...
Name: title, dtype: object
2328


In [25]:
########### load the whole embedding into memory ############
embeddings_index = dict()
f = open('glove.6B.300d.txt')
for num, line in enumerate(f):
    values = line.split()
    word = values[0]
    if word in stop_words :
#         print (word)
        continue
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    
f.close()
max_width = 300
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 399851 word vectors.


In [26]:
########### create a weight matrix for words in training docs ############
embedding_matrix = zeros((vocab_size, max_width))
mil=0
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        mil+=1
        embedding_matrix[i] = embedding_vector
print(mil)

2148


In [27]:
################ define model#################
model = Sequential()
e = Embedding(vocab_size, max_width, weights=[embedding_matrix], input_length=max_length, trainable=True )
model.add(e)
model.add(Conv1D(filters= 15 ,kernel_size= 3 ,activation="relu",strides=1))
model.add(Flatten())
model.add(Dense(15,  kernel_initializer="normal",activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

############## compile the model ##############
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc','binary_accuracy'])
############# summarize the model ##############
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 300)          698400    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 98, 15)            13515     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1470)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 15)                22065     
_________________________________________________________________
dropout_4 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 16        
Total params: 733,996
Trainable params: 733,996
Non-trainable params: 0
_________________________________________________________________
None

In [28]:
############## split into train test data ##############
padded_docs_train, padded_docs_test, labels_train, labels_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=69)
################### fit the model #####################
model.fit(padded_docs_train, labels_train, epochs=25, validation_split = 0.25,shuffle=True, verbose=1)

Train on 894 samples, validate on 299 samples
Epoch 1/25
894/894 [==============================] - 2s 3ms/step - loss: 0.6898 - acc: 0.5324 - binary_accuracy: 0.5324 - val_loss: 0.6826 - val_acc: 0.5886 - val_binary_accuracy: 0.5886
Epoch 2/25
894/894 [==============================] - 1s 904us/step - loss: 0.6350 - acc: 0.6409 - binary_accuracy: 0.6409 - val_loss: 0.6747 - val_acc: 0.6054 - val_binary_accuracy: 0.6054
Epoch 3/25
894/894 [==============================] - 1s 913us/step - loss: 0.5545 - acc: 0.7483 - binary_accuracy: 0.7483 - val_loss: 0.6643 - val_acc: 0.5886 - val_binary_accuracy: 0.5886
Epoch 4/25
894/894 [==============================] - 1s 921us/step - loss: 0.4377 - acc: 0.8289 - binary_accuracy: 0.8289 - val_loss: 0.6779 - val_acc: 0.6087 - val_binary_accuracy: 0.6087
Epoch 5/25
894/894 [==============================] - 1s 903us/step - loss: 0.3058 - acc: 0.9038 - binary_accuracy: 0.9038 - val_loss: 0.7343 - val_acc: 0.5953 - val_binary_accuracy: 0.5953
Epoch 

In [29]:
# evaluate the model
loss, accuracy,binaryAccuracy = model.evaluate(padded_docs_test, labels_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))
print('binaryAccuracy: %f' % (binaryAccuracy*100))
predicted = model.predict(padded_docs_test)

299/299 [==============================] - 0s 270us/step
Accuracy: 58.528428
binaryAccuracy: 58.528428


In [ ]:
predict_path = "../data/AXISBANKTest.csv"
test = pd.read_csv(predict_path)
############## prepare tokenizer #################
t1 = Tokenizer()
t1.fit_on_texts(test)

# predict_path = "../data/AXISBANKTest.csv"
# test = pd.read_csv(predict_path)
print(test.size)
########## integer encode the documents ##########
encoded_test = t.texts_to_sequences(test['title'])
print(len(encoded_test))
########## pad documents to a max length ##########
max_length = 100
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

predicted = model.predict(padded_test)
# for i in range(predicted.size):
#     if predicted[i] < 0.5 :
#         predicted[i] = 0.0
#     else :
#         predicted[i] = 1.0

In [ ]:
confidance = np.mean(predicted)
if confidance > 0.5:
    print ("buy")
    print("confidance is :",confidance)
    
else:
    if quantity != 0 :
        print ("sell")
        print("confidance is :",1-confidance)